#### import library

In [47]:
import nltk
import sklearn

print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The nltk version is 3.2.5.
The scikit-learn version is 0.21.3.


In [14]:
import os
os.chdir('D:\\Python for data science\\Practice\\NLP Project\\Text_classification')

import numpy as np
import re
import pickle
from sklearn.datasets import load_files
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

In [17]:
pwd

'D:\\Python for data science\\Practice\\NLP Project\\Text_classification'

#### Load Dataset

In [18]:
review = load_files('txt_sentoken\\')

X,y = review.data,review.target

#### Persisting the dataset

In [22]:
# Save these file on OS

with open('X.pickle','wb') as f:
    pickle.dump(X,f)
    

with open('y.pickle','wb') as f:
    pickle.dump(y,f)

In [53]:
# load these file from OS

with open('X.pickle','rb') as f:
    X = pickle.load(f)
    
with open('y.pickle','rb') as f:
    y = pickle.load(f)

In [54]:
#Preprocess dataset
corpus = []

for i in range(0,len(X)):
    review = re.sub(r'\W',' ',str(X[i])) # Remove preposition
    review = re.sub(r'\s+',' ',review) # Remove all large spaces
    review = re.sub(r'\s+[a-z]\s+',' ',review) #Remove single character word like i, a, 
    review = re.sub(r'^[a-z]\s+',' ',review)
    review = review.lower()
    corpus.append(review)


In [29]:
# #Create BOW Model
# from sklearn.feature_extraction.text import CountVectorizer
# vector = CountVectorizer(max_features=2000,min_df=3,max_df=.6,stop_words= stopwords.words('english'))
# x= vector.fit_transform(corpus).toarray()

# # Convert simple BOW to TF-IDF model
# from sklearn.feature_extraction.text import TfidfTransformer
# transform = TfidfTransformer()
# X = transform.fit_transform(x).toarray()

In [55]:
#alternatively we can we TFIDF directly
#Create BOW Model
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer(max_features=2000,min_df=3,max_df=.6,stop_words= stopwords.words('english'))
X= vector.fit_transform(corpus).toarray()

#Save this, as later this will use to predict the sentiment on unseen tweets

In [56]:
X.shape

(2000, 2000)

In [38]:
from sklearn.model_selection import train_test_split

text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size=0.2, random_state= 0)

In [39]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

classifier.fit(text_train,sent_train)

D:\Python\Installation\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [50]:
sent_pred = classifier.predict(text_test)

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

confusion_matrix(sent_pred,sent_test)

array([[168,  21],
       [ 40, 171]], dtype=int64)

In [51]:
accuracy_score(sent_pred,sent_test)

0.8475

In [52]:
# Save these model into OS
# Save these file on OS to predict the sentiment of unseen tweets 

with open('classifier.pickle','wb') as f:
    pickle.dump(classifier,f)


with open('Tfidf_vectorizer.pickle','wb') as f:
    pickle.dump(vector,f)

### Prediction on Unseen Corpus

In [65]:
#UnPack our pickle files
with open('classifier.pickle','rb') as f:  # classifier.pickle, is the file name stored in OS
    clf = pickle.load(f) #classifier is the variable name, to which it is now assigned
    
with open('Tfidf_vectorizer.pickle','rb') as f:
    Tfidf = pickle.load(f)

In [78]:
#sen = ['This is the worst approach for most users.']
sen = ["Don't get into a shouting match. If someone is aggressive or rude, respond with patience and kindness"]


In [79]:
sample = Tfidf.transform(sen).toarray()  # Fit to TFIDF vectorizer
result = clf.predict(sample) # predict using our classfier

In [80]:
if result[0] == 1:
    print("Sentence is showing positive behave")
else:
    print("Sentence is showing Nagative behave")

Sentence is showing Nagative behave
